# Baseline: Greedy Tree Packing with Fix Direction

This baseline implements:
1. Greedy tree placement with weighted angles
2. Fix direction rotation optimization
3. Score calculation and validation

In [1]:
import math
import random
import numpy as np
import pandas as pd
from decimal import Decimal, getcontext
from shapely import affinity
from shapely.geometry import Polygon
from shapely.ops import unary_union
from shapely.strtree import STRtree
from scipy.spatial import ConvexHull
from scipy.optimize import minimize_scalar
import time

# Set precision for Decimal
getcontext().prec = 25
scale_factor = Decimal('1e15')

print('Libraries loaded successfully')

Libraries loaded successfully


In [2]:
class ChristmasTree:
    """Represents a single, rotatable Christmas tree of a fixed size."""

    def __init__(self, center_x='0', center_y='0', angle='0'):
        """Initializes the Christmas tree with a specific position and rotation."""
        self.center_x = Decimal(str(center_x))
        self.center_y = Decimal(str(center_y))
        self.angle = Decimal(str(angle))

        trunk_w = Decimal('0.15')
        trunk_h = Decimal('0.2')
        base_w = Decimal('0.7')
        mid_w = Decimal('0.4')
        top_w = Decimal('0.25')
        tip_y = Decimal('0.8')
        tier_1_y = Decimal('0.5')
        tier_2_y = Decimal('0.25')
        base_y = Decimal('0.0')
        trunk_bottom_y = -trunk_h

        initial_polygon = Polygon(
            [
                # Start at Tip
                (Decimal('0.0') * scale_factor, tip_y * scale_factor),
                # Right side - Top Tier
                (top_w / Decimal('2') * scale_factor, tier_1_y * scale_factor),
                (top_w / Decimal('4') * scale_factor, tier_1_y * scale_factor),
                # Right side - Middle Tier
                (mid_w / Decimal('2') * scale_factor, tier_2_y * scale_factor),
                (mid_w / Decimal('4') * scale_factor, tier_2_y * scale_factor),
                # Right side - Bottom Tier
                (base_w / Decimal('2') * scale_factor, base_y * scale_factor),
                # Right Trunk
                (trunk_w / Decimal('2') * scale_factor, base_y * scale_factor),
                (trunk_w / Decimal('2') * scale_factor, trunk_bottom_y * scale_factor),
                # Left Trunk
                (-(trunk_w / Decimal('2')) * scale_factor, trunk_bottom_y * scale_factor),
                (-(trunk_w / Decimal('2')) * scale_factor, base_y * scale_factor),
                # Left side - Bottom Tier
                (-(base_w / Decimal('2')) * scale_factor, base_y * scale_factor),
                # Left side - Middle Tier
                (-(mid_w / Decimal('4')) * scale_factor, tier_2_y * scale_factor),
                (-(mid_w / Decimal('2')) * scale_factor, tier_2_y * scale_factor),
                # Left side - Top Tier
                (-(top_w / Decimal('4')) * scale_factor, tier_1_y * scale_factor),
                (-(top_w / Decimal('2')) * scale_factor, tier_1_y * scale_factor),
            ]
        )
        rotated = affinity.rotate(initial_polygon, float(self.angle), origin=(0, 0))
        self.polygon = affinity.translate(rotated,
                                          xoff=float(self.center_x * scale_factor),
                                          yoff=float(self.center_y * scale_factor))
    
    def clone(self):
        return ChristmasTree(
            center_x=str(self.center_x),
            center_y=str(self.center_y),
            angle=str(self.angle),
        )

print('ChristmasTree class defined')

ChristmasTree class defined


In [3]:
def generate_weighted_angle():
    """
    Generates a random angle with a distribution weighted by abs(sin(2*angle)).
    This helps place more trees in corners, and makes the packing less round.
    """
    while True:
        angle = random.uniform(0, 2 * math.pi)
        if random.uniform(0, 1) < abs(math.sin(2 * angle)):
            return angle

def initialize_trees(num_trees, existing_trees=None):
    """
    Builds a simple, greedy starting configuration.
    """
    if num_trees == 0:
        return [], Decimal('0')

    if existing_trees is None:
        placed_trees = []
    else:
        placed_trees = list(existing_trees)

    num_to_add = num_trees - len(placed_trees)

    if num_to_add > 0:
        unplaced_trees = [
            ChristmasTree(angle=random.uniform(0, 360)) for _ in range(num_to_add)]
        if not placed_trees:  # Only place the first tree at origin if starting from scratch
            placed_trees.append(unplaced_trees.pop(0))

        for tree_to_place in unplaced_trees:
            placed_polygons = [p.polygon for p in placed_trees]
            tree_index = STRtree(placed_polygons)

            best_px = None
            best_py = None
            min_radius = Decimal('Infinity')

            # This loop tries 10 random starting attempts and keeps the best one
            for _ in range(10):
                # The new tree starts at a position 20 from the center, at a random vector angle.
                angle = generate_weighted_angle()
                vx = Decimal(str(math.cos(angle)))
                vy = Decimal(str(math.sin(angle)))

                # Move towards center along the vector in steps of 0.5 until collision
                radius = Decimal('20.0')
                step_in = Decimal('0.5')

                collision_found = False
                while radius >= 0:
                    px = radius * vx
                    py = radius * vy

                    candidate_poly = affinity.translate(
                        tree_to_place.polygon,
                        xoff=float(px * scale_factor),
                        yoff=float(py * scale_factor))

                    # Looking for nearby objects
                    possible_indices = tree_index.query(candidate_poly)
                    # This is the collision detection step
                    if any((candidate_poly.intersects(placed_polygons[i]) and not
                            candidate_poly.touches(placed_polygons[i]))
                           for i in possible_indices):
                        collision_found = True
                        break
                    radius -= step_in

                # back up in steps of 0.05 until it no longer has a collision.
                if collision_found:
                    step_out = Decimal('0.05')
                    while True:
                        radius += step_out
                        px = radius * vx
                        py = radius * vy

                        candidate_poly = affinity.translate(
                            tree_to_place.polygon,
                            xoff=float(px * scale_factor),
                            yoff=float(py * scale_factor))

                        possible_indices = tree_index.query(candidate_poly)
                        if not any((candidate_poly.intersects(placed_polygons[i]) and not
                                   candidate_poly.touches(placed_polygons[i]))
                                   for i in possible_indices):
                            break
                else:
                    # No collision found even at the center. Place it at the center.
                    radius = Decimal('0')
                    px = Decimal('0')
                    py = Decimal('0')

                if radius < min_radius:
                    min_radius = radius
                    best_px = px
                    best_py = py

            tree_to_place.center_x = best_px
            tree_to_place.center_y = best_py
            tree_to_place.polygon = affinity.translate(
                tree_to_place.polygon,
                xoff=float(tree_to_place.center_x * scale_factor),
                yoff=float(tree_to_place.center_y * scale_factor),
            )
            placed_trees.append(tree_to_place)

    all_polygons = [t.polygon for t in placed_trees]
    bounds = unary_union(all_polygons).bounds

    minx = Decimal(bounds[0]) / scale_factor
    miny = Decimal(bounds[1]) / scale_factor
    maxx = Decimal(bounds[2]) / scale_factor
    maxy = Decimal(bounds[3]) / scale_factor

    width = maxx - minx
    height = maxy - miny
    side_length = max(width, height)

    return placed_trees, side_length

print('Greedy placement functions defined')

Greedy placement functions defined


In [4]:
# Fix direction optimization functions
def calculate_bbox_side_at_angle(angle_deg, points):
    angle_rad = np.radians(angle_deg)
    c, s = np.cos(angle_rad), np.sin(angle_rad)
    rot_matrix_T = np.array([[c, s], [-s, c]])
    rotated_points = points.dot(rot_matrix_T)
    min_xy = np.min(rotated_points, axis=0)
    max_xy = np.max(rotated_points, axis=0)
    return max(max_xy[0] - min_xy[0], max_xy[1] - min_xy[1])

def optimize_rotation(trees):
    """Find optimal rotation angle to minimize bounding box side length."""
    all_points = []
    for tree in trees:
        coords = list(tree.polygon.exterior.coords)
        for coord in coords:
            all_points.append([float(coord[0]) / float(scale_factor), 
                               float(coord[1]) / float(scale_factor)])
    points_np = np.array(all_points)

    hull_points = points_np[ConvexHull(points_np).vertices]

    initial_side = calculate_bbox_side_at_angle(0, hull_points)

    res = minimize_scalar(lambda a: calculate_bbox_side_at_angle(a, hull_points),
                          bounds=(0.001, 89.999), method='bounded')
    found_angle_deg = res.x
    found_side = res.fun

    improvement = initial_side - found_side
    EPSILON = 1e-8

    if improvement > EPSILON:
        best_angle_deg = found_angle_deg
        best_side = Decimal(str(found_side))
    else:
        best_angle_deg = 0.0
        best_side = Decimal(str(initial_side))

    return best_side, best_angle_deg

def apply_rotation(trees, angle_deg):
    """Apply rotation to all trees around the center of the configuration."""
    if not trees or abs(angle_deg) < 1e-9:
        return [t.clone() for t in trees]

    bounds = [t.polygon.bounds for t in trees]
    min_x = min(b[0] for b in bounds)
    min_y = min(b[1] for b in bounds)
    max_x = max(b[2] for b in bounds)
    max_y = max(b[3] for b in bounds)
    rotation_center = np.array([(min_x + max_x) / 2.0, (min_y + max_y) / 2.0])

    angle_rad = np.radians(angle_deg)
    c, s = np.cos(angle_rad), np.sin(angle_rad)
    rot_matrix = np.array([[c, -s], [s, c]])

    points = np.array([[float(t.center_x), float(t.center_y)] for t in trees])
    shifted = points - rotation_center / float(scale_factor)
    rotated = shifted.dot(rot_matrix.T) + rotation_center / float(scale_factor)

    rotated_trees = []
    for i in range(len(trees)):
        new_tree = ChristmasTree(
            center_x=str(rotated[i, 0]),
            center_y=str(rotated[i, 1]),
            angle=str(float(trees[i].angle) + angle_deg)
        )
        rotated_trees.append(new_tree)
    return rotated_trees

print('Fix direction functions defined')

Fix direction functions defined


In [5]:
def get_tree_list_side_length(tree_list):
    """Calculate the side length of the bounding square."""
    all_polygons = [t.polygon for t in tree_list]
    bounds = unary_union(all_polygons).bounds
    width = Decimal(bounds[2] - bounds[0]) / scale_factor
    height = Decimal(bounds[3] - bounds[1]) / scale_factor
    return max(width, height)

def has_overlap(trees):
    """Check if any two ChristmasTree polygons overlap."""
    if len(trees) <= 1:
        return False

    polygons = [t.polygon for t in trees]
    tree_index = STRtree(polygons)

    for i, poly in enumerate(polygons):
        indices = tree_index.query(poly)
        for idx in indices:
            if idx == i:
                continue
            if poly.intersects(polygons[idx]) and not poly.touches(polygons[idx]):
                return True
    return False

def calculate_total_score(dict_of_side_length):
    """Calculate total score from side lengths."""
    score = Decimal('0')
    for n, side in dict_of_side_length.items():
        score += side ** 2 / Decimal(str(n))
    return score

print('Scoring functions defined')

Scoring functions defined


In [6]:
# Build the index of the submission
index = [f'{n:03d}_{t}' for n in range(1, 201) for t in range(n)]
print(f'Total rows in submission: {len(index)}')

Total rows in submission: 20100


In [7]:
# Generate the greedy baseline solution
random.seed(42)  # For reproducibility

start_time = time.time()

all_tree_configs = {}  # n -> list of trees
all_side_lengths = {}  # n -> side length

current_placed_trees = []

for n in range(1, 201):
    current_placed_trees, side = initialize_trees(n, existing_trees=current_placed_trees)
    all_tree_configs[n] = [t.clone() for t in current_placed_trees]
    all_side_lengths[n] = side
    
    if n % 20 == 0:
        elapsed = time.time() - start_time
        print(f'n={n:3d}: side={float(side):.6f}, elapsed={elapsed:.1f}s')

greedy_score = calculate_total_score(all_side_lengths)
print(f'\nGreedy baseline score: {float(greedy_score):.6f}')
print(f'Total time: {time.time() - start_time:.1f}s')

n= 20: side=4.075499, elapsed=0.3s


n= 40: side=5.881665, elapsed=0.8s


n= 60: side=7.166137, elapsed=1.3s


n= 80: side=8.688082, elapsed=1.8s


n=100: side=9.476442, elapsed=2.5s


n=120: side=10.054136, elapsed=3.2s


n=140: side=10.948304, elapsed=4.1s


n=160: side=11.344507, elapsed=5.0s


n=180: side=12.127983, elapsed=6.0s


n=200: side=13.125883, elapsed=7.1s

Greedy baseline score: 178.919725
Total time: 7.1s


In [ ]:
# Apply fix_direction optimization to each configuration
print('Applying fix_direction optimization...')

optimized_configs = {}
optimized_side_lengths = {}

start_time = time.time()

for n in range(1, 201):
    trees = all_tree_configs[n]
    
    # Get current side length
    current_side = all_side_lengths[n]
    
    # Try to optimize rotation
    if len(trees) > 1:
        best_side, best_angle = optimize_rotation(trees)
        
        if best_side < current_side:
            # Apply the rotation
            rotated_trees = apply_rotation(trees, best_angle)
            optimized_configs[n] = rotated_trees
            optimized_side_lengths[n] = best_side
        else:
            optimized_configs[n] = [t.clone() for t in trees]
            optimized_side_lengths[n] = current_side
    else:
        optimized_configs[n] = [t.clone() for t in trees]
        optimized_side_lengths[n] = current_side
    
    if n % 20 == 0:
        elapsed = time.time() - start_time
        print(f'n={n:3d}: side={float(optimized_side_lengths[n]):.6f}, elapsed={elapsed:.1f}s')

optimized_score = calculate_total_score(optimized_side_lengths)
print(f'\nOptimized score: {float(optimized_score):.6f}')
print(f'Improvement: {float(greedy_score - optimized_score):.6f}')

In [ ]:
# Validate for overlaps
print('Validating for overlaps...')

overlap_configs = []
for n in range(1, 201):
    trees = optimized_configs[n]
    if has_overlap(trees):
        overlap_configs.append(n)
        print(f'  Overlap detected in n={n}')

if overlap_configs:
    print(f'\n❌ Overlaps found in {len(overlap_configs)} configurations: {overlap_configs}')
else:
    print('\n✅ No overlaps detected!')

In [ ]:
# Create submission DataFrame
tree_data = []

for n in range(1, 201):
    trees = optimized_configs[n]
    for tree in trees:
        tree_data.append([float(tree.center_x), float(tree.center_y), float(tree.angle)])

cols = ['x', 'y', 'deg']
submission = pd.DataFrame(index=index, columns=cols, data=tree_data).rename_axis('id')

# Round to 6 decimal places
for col in cols:
    submission[col] = submission[col].astype(float).round(decimals=6)

# Prepend 's' to ensure string format
for col in submission.columns:
    submission[col] = 's' + submission[col].astype('string')

print(submission.head(20))
print(f'\nTotal rows: {len(submission)}')

In [ ]:
# Save submission
submission.to_csv('/home/submission/submission.csv')
submission.to_csv('/home/code/experiments/001_baseline/submission.csv')
print('Submission saved!')
print(f'\nFinal score: {float(optimized_score):.6f}')